<a href="https://colab.research.google.com/github/FuriouStyles/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/Stephen_P_assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [6]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [7]:
df = df[(df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000)]
print(min(df['SALE_PRICE']), max(df['SALE_PRICE']))

100044 1999877


In [0]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)

In [0]:
cutoff = pd.to_datetime('04/01/2019')

In [0]:
train = df[df['SALE_DATE'] < cutoff]
test = df[df['SALE_DATE'] >= cutoff]

In [11]:
train.shape, test.shape

((10442, 21), (2969, 21))

In [12]:
df.nunique()

BOROUGH                               5
NEIGHBORHOOD                         11
BUILDING_CLASS_CATEGORY              41
TAX_CLASS_AT_PRESENT                 10
BLOCK                              5980
LOT                                1160
EASE-MENT                             0
BUILDING_CLASS_AT_PRESENT           105
ADDRESS                           13346
APARTMENT_NUMBER                   1018
ZIP_CODE                            183
RESIDENTIAL_UNITS                    32
COMMERCIAL_UNITS                     18
TOTAL_UNITS                          37
LAND_SQUARE_FEET                   2299
GROSS_SQUARE_FEET                  2485
YEAR_BUILT                          131
TAX_CLASS_AT_TIME_OF_SALE             3
BUILDING_CLASS_AT_TIME_OF_SALE      105
SALE_PRICE                         2764
SALE_DATE                           103
dtype: int64

In [0]:
high_card_features = ['BUILDING_CLASS_CATEGORY', 'BLOCK', 'LOT', 'EASE-MENT', 'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'BUILDING_CLASS_AT_TIME_OF_SALE', 'SALE_PRICE', 'SALE_DATE'] #EASE-MENT has zero cardinality so we'll drop it together with the rest of the high cardinality features because it's essentially useless
target = 'SALE_PRICE'

features = train.columns.drop([target] + high_card_features)

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [0]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [15]:
X_train.shape

(10442, 27)

In [16]:
X_train.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_1,BOROUGH_2,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_UPPER EAST SIDE (79-96),NEIGHBORHOOD_UPPER WEST SIDE (59-79),NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_ASTORIA,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_GRAMERCY,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_FOREST HILLS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_2,TAX_CLASS_AT_PRESENT_4,TAX_CLASS_AT_PRESENT_2A,TAX_CLASS_AT_PRESENT_1B,TAX_CLASS_AT_PRESENT_1A,TAX_CLASS_AT_PRESENT_2C,TAX_CLASS_AT_PRESENT_1C,TAX_CLASS_AT_PRESENT_2B,TAX_CLASS_AT_PRESENT_1D,TAX_CLASS_AT_TIME_OF_SALE
44,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
61,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
66,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2
67,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2
71,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2


In [17]:
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for i in range(1, len(X_train.columns) + 1):
  selector = SelectKBest(score_func=f_regression, k=i)
  model = LinearRegression()

  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)
  mae = mean_absolute_error(y_test, y_pred)
  print(f'Test MAE with {i} features: {mae:.0f}')

Test MAE with 1 features: 317953
Test MAE with 2 features: 314520
Test MAE with 3 features: 304732
Test MAE with 4 features: 305201
Test MAE with 5 features: 302822
Test MAE with 6 features: 303006
Test MAE with 7 features: 302714
Test MAE with 8 features: 282662
Test MAE with 9 features: 282531
Test MAE with 10 features: 282001
Test MAE with 11 features: 281676
Test MAE with 12 features: 281138
Test MAE with 13 features: 280794
Test MAE with 14 features: 280660
Test MAE with 15 features: 280727
Test MAE with 16 features: 280664
Test MAE with 17 features: 280823
Test MAE with 18 features: 280630
Test MAE with 19 features: 280722
Test MAE with 20 features: 279714
Test MAE with 21 features: 276294
Test MAE with 22 features: 276310
Test MAE with 23 features: 276336
Test MAE with 24 features: 276009
Test MAE with 25 features: 276024
Test MAE with 26 features: 275981
Test MAE with 27 features: 276053


In [0]:
from sklearn.preprocessing import scale

X_train_temp = scale(X_train)
X_test_temp = scale(X_test)
y_train_temp = scale(y_train)
y_test_temp = scale(y_test)

In [19]:
X_train_scaled = pd.DataFrame(X_train_temp, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_temp, columns=X_test.columns)
y_train_scaled = pd.DataFrame(y_train_temp)
y_test_scaled = pd.DataFrame(y_test_temp)
X_train_scaled

,BOROUGH_3,BOROUGH_4,BOROUGH_1,BOROUGH_2,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_UPPER EAST SIDE (79-96),NEIGHBORHOOD_UPPER WEST SIDE (59-79),NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_EAST NEW YORK,NEIGHBORHOOD_ASTORIA,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_GRAMERCY,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_FOREST HILLS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_2,TAX_CLASS_AT_PRESENT_4,TAX_CLASS_AT_PRESENT_2A,TAX_CLASS_AT_PRESENT_1B,TAX_CLASS_AT_PRESENT_1A,TAX_CLASS_AT_PRESENT_2C,TAX_CLASS_AT_PRESENT_1C,TAX_CLASS_AT_PRESENT_2B,TAX_CLASS_AT_PRESENT_1D,TAX_CLASS_AT_TIME_OF_SALE
0,1.662600,-0.726601,-0.462497,-0.341573,-0.348179,0.430244,-0.135404,-0.116152,-0.132814,-0.116574,-0.111841,-0.174043,-0.088967,-0.1343,-0.098828,-0.132065,1.053528,-0.814576,-0.203186,-0.143609,-0.096832,-0.142214,-0.178377,-0.046984,-0.046984,-0.054568,-0.821978
1,-0.601468,1.376272,-0.462497,-0.341573,-0.348179,0.430244,-0.135404,-0.116152,-0.132814,-0.116574,-0.111841,-0.174043,-0.088967,-0.1343,-0.098828,-0.132065,1.053528,-0.814576,-0.203186,-0.143609,-0.096832,-0.142214,-0.178377,-0.046984,-0.046984,-0.054568,-0.821978
2,-0.601468,-0.726601,2.162175,-0.341573,-0.348179,0.430244,-0.135404,-0.116152,-0.132814,-0.116574,-0.111841,-0.174043,-0.088967,-0.1343,-0.098828,-0.132065,-0.949192,1.227633,-0.203186,-0.143609,-0.096832,-0.142214,-0.178377,-0.046984,-0.046984,-0.054568,0.617173
3,-0.601468,-0.726601,2.162175,-0.341573,-0.348179,-2.324265,7.385293,-0.116152,-0.132814,-0.116574,-0.111841,-0.174043,-0.088967,-0.1343,-0.098828,-0.132065,-0.949192,1.227633,-0.203186,-0.143609,-0.096832,-0.142214,-0.178377,-0.046984,-0.046984,-0.054568,0.617173
4,-0.601468,-0.726601,2.162175,-0.341573,-0.348179,-2.324265,-0.135404,8.609431,-0.132814,-0.116574,-0.111841,-0.174043,-0.088967,-0.1343,-0.098828,-0.132065,-0.949192,1.227633,-0.203186,-0.143609,-0.096832,-0.142214,-0.178377,-0.046984,-0.046984,-0.054568,0.617173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10437,-0.601468,-0.726601,-0.462497,-0.341573,2.872089,0.430244,-0.135404,-0.116152,-0.132814,-0.116574,-0.111841,-0.174043,-0.088967,-0.1343,-0.098828,-0.132065,1.053528,-0.814576,-0.203186,-0.143609,-0.096832,-0.142214,-0.178377,-0.046984,-0.046984,-0.054568,-0.821978
10438,-0.601468,-0.726601,-0.462497,-0.341573,2.872089,0.430244,-0.135404,-0.116152,-0.132814,-0.116574,-0.111841,-0.174043,-0.088967,-0.1343,-0.098828,-0.132065,1.053528,-0.814576,-0.203186,-0.143609,-0.096832,-0.142214,-0.178377,-0.046984,-0.046984,-0.054568,-0.821978
10439,-0.601468,-0.726601,-0.462497,-0.341573,2.872089,0.430244,-0.135404,-0.116152,-0.132814,-0.116574,-0.111841,-0.174043,-0.088967,-0.1343,-0.098828,-0.132065,1.053528,-0.814576,-0.203186,-0.143609,-0.096832,-0.142214,-0.178377,-0.046984,-0.046984,-0.054568,-0.821978
10440,-0.601468,-0.726601,-0.462497,2.927633,-0.348179,0.430244,-0.135404,-0.116152,-0.132814,-0.116574,-0.111841,-0.174043,-0.088967,-0.1343,-0.098828,-0.132065,1.053528,-0.814576,-0.203186,-0.143609,-0.096832,-0.142214,-0.178377,-0.046984,-0.046984,-0.054568,-0.821978


In [21]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

alpha_mse = {}

for alpha in range(0, 200, 1):
  ridge_reg_split = Ridge(alpha=alpha).fit(X_train_scaled, y_train_scaled)
  mse = mean_squared_error(y_test_scaled, ridge_reg_split.predict(X_test_scaled))
  alpha_mse[alpha] = mse

print(sorted(alpha_mse.items(), reverse=True))

[(199, 0.7624180816838206), (198, 0.762422720976874), (197, 0.7624273726015773), (196, 0.7624320365678612), (195, 0.7624367128856118), (194, 0.7624414015646707), (193, 0.7624461026148318), (192, 0.7624508160458414), (191, 0.7624555418673951), (190, 0.762460280089137), (189, 0.7624650307206581), (188, 0.7624697937714936), (187, 0.7624745692511221), (186, 0.7624793571689634), (185, 0.762484157534376), (184, 0.762488970356656), (183, 0.762493795645034), (182, 0.7624986334086741), (181, 0.7625034836566706), (180, 0.7625083463980463), (179, 0.7625132216417503), (178, 0.7625181093966552), (177, 0.7625230096715545), (176, 0.76252792247516), (175, 0.7625328478160995), (174, 0.7625377857029139), (173, 0.7625427361440537), (172, 0.7625476991478765), (171, 0.7625526747226441), (170, 0.7625576628765192), (169, 0.7625626636175614), (168, 0.7625676769537244), (167, 0.762572702892853), (166, 0.7625777414426774), (165, 0.7625827926108125), (164, 0.7625878564047504), (163, 0.7625929328318594), (162, 0.